In [27]:
import os,sys

#stats
import numpy as np

from sklearn.model_selection import train_test_split, StratifiedKFold \
                                    , cross_val_score ,cross_val_predict
from sklearn.base import clone
from sklearn.linear_model import SGDClassifier, LogisticRegression
from sklearn.metrics import confusion_matrix, precision_score, recall_score \
                            ,precision_recall_curve, roc_curve, accuracy_score
from sklearn.pipeline import Pipeline
from sklearn.grid_search import GridSearchCV, RandomizedSearchCV

from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC, LinearSVC
from sklearn.neighbors import KNeighborsClassifier, RadiusNeighborsClassifier

from sklearn.preprocessing import StandardScaler

from scipy.ndimage.interpolation import shift

from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.naive_bayes import GaussianNB

# data
import sklearn.datasets
from sklearn.datasets import fetch_mldata #mldata.org is down, don't use
from tensorflow.examples.tutorials.mnist import input_data
import pandas as pd

#graphs 
import matplotlib
import matplotlib.pyplot as plt

#some variables
seed = 42 #random seed
np.random.seed(seed)

#display
from IPython.display import HTML

#magic
%matplotlib  inline

In [28]:
titan_train = pd.read_csv("{}/mldata/titanic/train_proc_v1.csv".format(os.getcwd()),
                         index_col = 0)

In [29]:
HTML(titan_train.head().to_html())

,PassengerId,Survived,Pclass,Name,Age,SibSp,Parch,Ticket,Fare,Cabin,sex_bin,Cherbourg,Queenstown,Southampton,room,deck_A,deck_B,deck_C,deck_D,deck_E,deck_F,deck_G,deck_T,has_room,family,title_Don,title_Dona,title_Major,title_Capt,title_Mme,title_Dr,title_Mr,title_Col,title_Jonkheer,title_Mrs,title_Mlle,title_Miss,title_Rev,title_Lady,title_Sir,title_the Countess
0,1,0,3,"Braund, Mr. Owen Harris",22.0,1,0,A/5 21171,7.2500,NaN,1,0,0,1,NaN,0,0,0,0,0,0,0,0,False,1,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",38.0,1,0,PC 17599,71.2833,C85,0,1,0,0,85.0,0,0,1,0,0,0,0,0,True,1,False,False,False,False,False,False,True,False,False,True,False,False,False,False,False,False
2,3,1,3,"Heikkinen, Miss. Laina",26.0,0,0,STON/O2. 3101282,7.9250,NaN,0,0,0,1,NaN,0,0,0,0,0,0,0,0,False,0,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",35.0,1,0,113803,53.1000,C123,0,0,0,1,123.0,0,0,1,0,0,0,0,0,True,1,False,False,False,False,False,False,True,False,False,True,False,False,False,False,False,False
4,5,0,3,"Allen, Mr. William Henry",35.0,0,0,373450,8.0500,NaN,1,0,0,1,NaN,0,0,0,0,0,0,0,0,False,0,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False


In [30]:
y_train = titan_train["Survived"].values

these have nulls

In [31]:
null_vals = ["Survived", "Name", "Ticket", "Cabin", "room"]

In [32]:
X_train = titan_train.drop(null_vals, axis = 1).values

In [33]:
titan_train.drop(to_drop, axis = 1).isnull().sum()

PassengerId           0
Pclass                0
Age                   0
SibSp                 0
Parch                 0
Fare                  0
sex_bin               0
Cherbourg             0
Queenstown            0
Southampton           0
deck_A                0
deck_B                0
deck_C                0
deck_D                0
deck_E                0
deck_F                0
deck_G                0
deck_T                0
has_room              0
family                0
title_Don             0
title_Dona            0
title_Major           0
title_Capt            0
title_Mme             0
title_Dr              0
title_Mr              0
title_Col             0
title_Jonkheer        0
title_Mrs             0
title_Mlle            0
title_Miss            0
title_Rev             0
title_Lady            0
title_Sir             0
title_the Countess    0
dtype: int64

In [34]:
scaler = StandardScaler()

X = scaler.fit_transform(X_train)

/home/edd/.local/share/virtualenvs/ml_projects-PtpjDyvs/lib/python3.6/site-packages/sklearn/utils/validation.py:475: DataConversionWarning: Data with input dtype object was converted to float64 by StandardScaler.
  warnings.warn(msg, DataConversionWarning)


In [35]:
log_clf = LogisticRegression()
scores = cross_val_score(log_clf, X, y_train, cv = 3, scoring = "accuracy")

In [36]:
print("logistic regression accuracy scores: {}\n\
       logistic regression accuracy mean: {}\n\
       logistic regression accuracy devation: {}".format(
                np.round(scores, decimals = 3),
                np.round(np.mean(scores), decimals = 3),
                np.round(np.std(scores), decimals = 3)
    ))

logistic regression accuracy scores: [0.791 0.805 0.838]
       logistic regression accuracy mean: 0.811
       logistic regression accuracy devation: 0.02


Gridsearch with log regression

In [49]:
# Create regularization penalty space
penalty = ['l1', 'l2']

# Create regularization hyperparameter space
C = np.logspace(0, 4, 10)

# Create hyperparameter options
hyperparameters = dict(C=C, penalty=penalty)

In [51]:
log_grd = GridSearchCV(log_clf, hyperparameters, cv=3, verbose=3)

In [53]:
best_mdl = log_grd.fit(X, y_train)

Fitting 3 folds for each of 20 candidates, totalling 60 fits
[CV] C=1.0, penalty=l1 ...............................................
[CV] ...................... C=1.0, penalty=l1, score=0.794613 -   0.0s
[CV] C=1.0, penalty=l1 ...............................................
[CV] ...................... C=1.0, penalty=l1, score=0.808081 -   0.0s
[CV] C=1.0, penalty=l1 ...............................................
[CV] ...................... C=1.0, penalty=l1, score=0.831650 -   0.0s
[CV] C=1.0, penalty=l2 ...............................................
[CV] ...................... C=1.0, penalty=l2, score=0.791246 -   0.0s
[CV] C=1.0, penalty=l2 ...............................................
[CV] ...................... C=1.0, penalty=l2, score=0.804714 -   0.0s
[CV] C=1.0, penalty=l2 ...............................................
[CV] ...................... C=1.0, penalty=l2, score=0.838384 -   0.0s
[CV] C=2.7825594022071245, penalty=l1 ................................
[CV] ....... C=2

[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.0s remaining:    0.0s


[CV] ........ C=7.742636826811269, penalty=l2, score=0.801347 -   0.0s
[CV] C=7.742636826811269, penalty=l2 .................................
[CV] ........ C=7.742636826811269, penalty=l2, score=0.835017 -   0.0s
[CV] C=21.544346900318832, penalty=l1 ................................
[CV] ....... C=21.544346900318832, penalty=l1, score=0.787879 -   0.1s
[CV] C=21.544346900318832, penalty=l1 ................................
[CV] ....... C=21.544346900318832, penalty=l1, score=0.801347 -   0.1s
[CV] C=21.544346900318832, penalty=l1 ................................
[CV] ....... C=21.544346900318832, penalty=l1, score=0.835017 -   0.0s
[CV] C=21.544346900318832, penalty=l2 ................................
[CV] ....... C=21.544346900318832, penalty=l2, score=0.791246 -   0.0s
[CV] C=21.544346900318832, penalty=l2 ................................
[CV] ....... C=21.544346900318832, penalty=l2, score=0.801347 -   0.0s
[CV] C=21.544346900318832, penalty=l2 ................................
[CV] .

[Parallel(n_jobs=1)]: Done  60 out of  60 | elapsed:    2.7s finished


In [54]:
print('Best Penalty:', best_mdl.best_estimator_.get_params()['penalty'])
print('Best C:', best_mdl.best_estimator_.get_params()['C'])

Best Penalty: l1
Best C: 1.0


In [55]:
scores = cross_val_score(log_grd, X, y_train, cv = 3, scoring = "accuracy")

Fitting 3 folds for each of 20 candidates, totalling 60 fits
[CV] C=1.0, penalty=l1 ...............................................
[CV] ...................... C=1.0, penalty=l1, score=0.808081 -   0.0s
[CV] C=1.0, penalty=l1 ...............................................
[CV] ...................... C=1.0, penalty=l1, score=0.792929 -   0.0s
[CV] C=1.0, penalty=l1 ...............................................
[CV] ...................... C=1.0, penalty=l1, score=0.843434 -   0.0s
[CV] C=1.0, penalty=l2 ...............................................
[CV] ...................... C=1.0, penalty=l2, score=0.797980 -   0.0s
[CV] C=1.0, penalty=l2 ...............................................
[CV] ...................... C=1.0, penalty=l2, score=0.797980 -   0.0s
[CV] C=1.0, penalty=l2 ...............................................
[CV] ...................... C=1.0, penalty=l2, score=0.843434 -   0.0s
[CV] C=2.7825594022071245, penalty=l1 ................................
[CV] ....... C=2

[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.0s remaining:    0.0s


[CV] ....... C=21.544346900318832, penalty=l2, score=0.797980 -   0.0s
[CV] C=21.544346900318832, penalty=l2 ................................
[CV] ....... C=21.544346900318832, penalty=l2, score=0.792929 -   0.0s
[CV] C=21.544346900318832, penalty=l2 ................................
[CV] ....... C=21.544346900318832, penalty=l2, score=0.838384 -   0.0s
[CV] C=59.94842503189409, penalty=l1 .................................
[CV] ........ C=59.94842503189409, penalty=l1, score=0.797980 -   0.1s
[CV] C=59.94842503189409, penalty=l1 .................................
[CV] ........ C=59.94842503189409, penalty=l1, score=0.797980 -   0.0s
[CV] C=59.94842503189409, penalty=l1 .................................
[CV] ........ C=59.94842503189409, penalty=l1, score=0.833333 -   0.0s
[CV] C=59.94842503189409, penalty=l2 .................................
[CV] ........ C=59.94842503189409, penalty=l2, score=0.797980 -   0.0s
[CV] C=59.94842503189409, penalty=l2 .................................
[CV] .

[Parallel(n_jobs=1)]: Done  60 out of  60 | elapsed:    1.2s finished
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.0s remaining:    0.0s


[CV] ........ C=7.742636826811269, penalty=l2, score=0.757576 -   0.0s
[CV] C=7.742636826811269, penalty=l2 .................................
[CV] ........ C=7.742636826811269, penalty=l2, score=0.833333 -   0.0s
[CV] C=21.544346900318832, penalty=l1 ................................
[CV] ....... C=21.544346900318832, penalty=l1, score=0.737374 -   0.0s
[CV] C=21.544346900318832, penalty=l1 ................................
[CV] ....... C=21.544346900318832, penalty=l1, score=0.772727 -   0.1s
[CV] C=21.544346900318832, penalty=l1 ................................
[CV] ....... C=21.544346900318832, penalty=l1, score=0.838384 -   0.0s
[CV] C=21.544346900318832, penalty=l2 ................................
[CV] ....... C=21.544346900318832, penalty=l2, score=0.737374 -   0.0s
[CV] C=21.544346900318832, penalty=l2 ................................
[CV] ....... C=21.544346900318832, penalty=l2, score=0.767677 -   0.0s
[CV] C=21.544346900318832, penalty=l2 ................................
[CV] .

[Parallel(n_jobs=1)]: Done  60 out of  60 | elapsed:    2.2s finished
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.0s remaining:    0.0s


[CV] ....... C=21.544346900318832, penalty=l1, score=0.782828 -   0.0s
[CV] C=21.544346900318832, penalty=l2 ................................
[CV] ....... C=21.544346900318832, penalty=l2, score=0.803030 -   0.0s
[CV] C=21.544346900318832, penalty=l2 ................................
[CV] ....... C=21.544346900318832, penalty=l2, score=0.808081 -   0.0s
[CV] C=21.544346900318832, penalty=l2 ................................
[CV] ....... C=21.544346900318832, penalty=l2, score=0.782828 -   0.0s
[CV] C=59.94842503189409, penalty=l1 .................................
[CV] ........ C=59.94842503189409, penalty=l1, score=0.797980 -   0.0s
[CV] C=59.94842503189409, penalty=l1 .................................
[CV] ........ C=59.94842503189409, penalty=l1, score=0.813131 -   0.0s
[CV] C=59.94842503189409, penalty=l1 .................................
[CV] ........ C=59.94842503189409, penalty=l1, score=0.782828 -   0.1s
[CV] C=59.94842503189409, penalty=l2 .................................
[CV] .

[Parallel(n_jobs=1)]: Done  60 out of  60 | elapsed:    1.6s finished


In [56]:
print("logistic regression grid search accuracy scores: {}\n\
       logistic regression grid search accuracy mean: {}\n\
       logistic regression grid search accuracy devation: {}".format(
                np.round(scores, decimals = 3),
                np.round(np.mean(scores), decimals = 3),
                np.round(np.std(scores), decimals = 3)
    ))

logistic regression grid search accuracy scores: [0.795 0.808 0.832]
       logistic regression grid search accuracy mean: 0.811
       logistic regression grid search accuracy devation: 0.015


In [37]:
sgd_clf =  SGDClassifier(random_state = seed, max_iter = 100)
scores = cross_val_score(sgd_clf, X, y_train, cv = 3, scoring = "accuracy")

In [38]:
print("SGD accuracy scores: {}\n\
       SGD accuracy mean: {}\n\
       SGD accuracy devation: {}".format(
                np.round(scores, decimals = 3),
                np.round(np.mean(scores), decimals = 3),
                np.round(np.std(scores), decimals = 3)
    ))

SGD accuracy scores: [0.764 0.761 0.815]
       SGD accuracy mean: 0.78
       SGD accuracy devation: 0.025


In [39]:
knn_clf = KNeighborsClassifier()

scores = cross_val_score(knn_clf, X, y_train, cv = 3, scoring = "accuracy")

In [40]:
print("KNN accuracy scores: {}\n\
       KNN accuracy mean: {}\n\
       KNN accuracy devation: {}".format(
                np.round(scores, decimals = 3),
                np.round(np.mean(scores), decimals = 3),
                np.round(np.std(scores), decimals = 3)
    ))

KNN accuracy scores: [0.778 0.818 0.828]
       KNN accuracy mean: 0.808
       KNN accuracy devation: 0.022


In [41]:
sv_clf = SVC()

scores = cross_val_score(sv_clf, X, y_train, cv = 3, scoring = "accuracy")

In [42]:
print("sv accuracy scores: {}\n\
       sv accuracy mean: {}\n\
       sv regression accuracy devation: {}".format(
                np.round(scores, decimals = 3),
                np.round(np.mean(scores), decimals = 3),
                np.round(np.std(scores), decimals = 3)
    ))


sv accuracy scores: [0.795 0.825 0.818]
       sv accuracy mean: 0.813
       sv regression accuracy devation: 0.013


In [43]:
rnf_clf = RandomForestClassifier()

scores = cross_val_score(rnf_clf, X, y_train, cv = 3, scoring = "accuracy")

In [44]:
print("Random Forest accuracy scores: {}\n\
       Random Forest accuracy mean: {}\n\
       Random Forest accuracy devation: {}".format(
                np.round(scores, decimals = 3),
                np.round(np.mean(scores), decimals = 3),
                np.round(np.std(scores), decimals = 3)
    ))


Random Forest accuracy scores: [0.785 0.788 0.788]
       Random Forest accuracy mean: 0.787
       Random Forest accuracy devation: 0.002


In [45]:
adb_clf = AdaBoostClassifier()

scores = cross_val_score(adb_clf, X, y_train, cv = 3, scoring = "accuracy")

In [46]:
print("adb accuracy scores: {}\n\
       adb accuracy mean: {}\n\
       adb accuracy devation: {}".format(
                np.round(scores, decimals = 3),
                np.round(np.mean(scores), decimals = 3),
                np.round(np.std(scores), decimals = 3)
    ))


adb accuracy scores: [0.616 0.788 0.616]
       adb accuracy mean: 0.673
       adb accuracy devation: 0.081
